In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[9],7
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 3.2e-07}","{'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.066328                    1 -0.005924
0.000750        2 -0.060461                    2 -0.002428
0.001052        3 -0.047631                    3  0.005668
0.001476        4 -0.033380                    4  0.014484
0.002070        5 -0.017776                    5  0.024240
0.002904        6 -0.000730                    6  0.034956
0.004074        7  0.017819                    7  0.046670
0.005714        8  0.037991                    8  0.059588
0.008015        9  0.059865                    9  0.073721
0.011243       10  0.083531                   10  0.089167
0.015771       11  0.109058                   11  0.106027
0.022122       12  0.136502                   12  0.124403
0.031031       13  0.165792                   13  0.144315
0.043528       14  0.192936                   14  0.163291
0.061057       15  0.213790                   15  0.178860
0.085645       16  0.231293                   16  0.192967
0.120136       17  0.248479                   17  0.207624
0.168516       18  0.265355                   18  0.222896
0.236378       19  0.281956                   19  0.238702
0.331549       20  0.298677                   20  0.255168
0.465100       21  0.313966                   21  0.270399
0.652400       22  0.287321                   22  0.256471
0.915100       23  0.212346                   23  0.207007
1.283650       24  0.140092                   24  0.154423
1.800600       25  0.087034                   25  0.109933
2.525700       26  0.050197                   26  0.072917
3.542800       27  0.025894                   27  0.043081
4.969550       28  0.013512                   28  0.022856
6.970850       29  0.010056                   29  0.012453
9.778100       30  0.008907                   30  0.007223
13.715850      31  0.007261                   31  0.004128
19.239350      32  0.005019                   32  0.002193
26.987250      33  0.004521                   33  0.002619
37.855300      34  0.005973                   34  0.004751
53.100050      35  0.006915                   35  0.006116
73.887500      36  0.007257                   36  0.006542
97.662500      37  0.006990                   37  0.006059
121.437500     38  0.005345                   38  0.004160
145.212500     39  0.003023                   39  0.001492
168.987500     40  0.000702                   40 -0.001371
192.762500     41 -0.001800                   41 -0.004506
216.537500     42 -0.004669                   42 -0.008025
240.312500     43 -0.008044                   43 -0.011937
264.087500     44 -0.012244                   44 -0.016381
287.862500     45 -0.017333                   45 -0.021103
311.637500     46 -0.019732                   46 -0.023088
335.412500     47 -0.016985                   47 -0.020465
359.187500     48 -0.012151                   48 -0.015611
382.962500     49 -0.006934                   49 -0.010144
406.737500     50 -0.001017                   50 -0.003930
430.512500     51  0.005343                   51  0.002796
454.287500     52  0.012053                   52  0.009610
478.062500     53  0.019007                   53  0.016557
501.837500     54  0.026127                   54  0.023801
525.612500     55  0.033322                   55  0.031556
549.387500     56  0.040446                   56  0.039353
573.162500     57  0.047340                   57  0.046606
596.937500     58  0.053781                   58  0.053075
620.712500     59  0.059557                   59  0.058677
644.487500     60  0.064495                   60  0.064047
668.262500     61  0.068514                   61  0.068746
692.037500     62  0.071656                   62  0.072887
715.812500     63  0.074078                   63  0.076415
739.587500     64  0.076069                   64  0.079406
763.362500     65  0.078006                   65  0.082093
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -6.887330  0.000000 -6.887330 -6.854929  9.401526e-08   
0.000624    2     -6.887338  0.000004 -6.887335 -6.854932  2.577534e-06   
0.000876    3     -6.887342  0.000006 -6.887336 -6.854934  3.638156e-06   
0.001229    4     -6.887346  0.000008 -6.887338 -6.854935  5.298439e-06   
0.001723    5     -6.887353  0.000012 -6.887340 -6.854937  7.887463e-06   
0.002417    6     -6.887360  0.000019 -6.887342 -6.854939  1.192288e-05   
0.003391    7     -6.887370  0.000028 -6.887342 -6.854941  1.820675e-05   
0.004757    8     -6.887383  0.000044 -6.887339 -6.854943  2.797731e-05   
0.006672    9     -6.887398  0.000067 -6.887330 -6.854945  4.316000e-05   
0.009359    10    -6.887415  0.000104 -6.887311 -6.854945  6.672322e-05   
0.013128    11    -6.887435  0.000161 -6.887274 -6.854942  1.032460e-04   
0.018415    12    -6.887454  0.000248 -6.887206 -6.854932  1.597816e-04   
0.025830    13    -6.887468  0.000382 -6.887086 -6.854910  2.471734e-04   
0.036232    14    -6.887468  0.000587 -6.886882 -6.854867  3.820083e-04   
0.050823    15    -6.887444  0.000896 -6.886548 -6.854790  5.874033e-04   
0.071291    16    -6.887380  0.001350 -6.886030 -6.854663  8.939562e-04   
0.100000    17    -6.887254  0.002011 -6.885243 -6.854459  1.346914e-03   
0.140271    18    -6.887021  0.002963 -6.884058 -6.854137  2.014945e-03   
0.196760    19    -6.886610  0.004328 -6.882282 -6.853628  2.998100e-03   
0.275997    20    -6.885901  0.006265 -6.879636 -6.852830  4.440564e-03   
0.387100    21    -6.884693  0.008988 -6.875705 -6.851579  6.548446e-03   
0.543100    22    -6.882651  0.012748 -6.869903 -6.849634  9.600258e-03   
0.761700    23    -6.879770  0.017307 -6.862463 -6.846991  1.359950e-02   
1.068500    24    -6.876573  0.021828 -6.854746 -6.844014  1.814599e-02   
1.498800    25    -6.873366  0.025761 -6.847605 -6.840926  2.293062e-02   
2.102400    26    -6.870251  0.028869 -6.841382 -6.837838  2.770312e-02   
2.949000    27    -6.867379  0.031031 -6.836348 -6.834918  3.209652e-02   
4.136600    28    -6.865026  0.032321 -6.832705 -6.832434  3.567426e-02   
5.802500    29    -6.863343  0.033305 -6.830039 -6.830540  3.829062e-02   
8.139200    30    -6.862173  0.034918 -6.827255 -6.829154  4.035250e-02   
11.417000   31    -6.861585  0.037788 -6.823797 -6.828442  4.244483e-02   
16.014700   32    -6.862187  0.042345 -6.819842 -6.829127  4.537845e-02   
22.464000   33    -6.865325  0.049317 -6.816008 -6.832581  5.050734e-02   
31.510500   34    -6.872055  0.060892 -6.811163 -6.839940  6.067391e-02   
44.200100   35    -6.882631  0.080446 -6.802185 -6.851859  7.973430e-02   
62.000000   36    -6.898493  0.110889 -6.787605 -6.870452  1.112259e-01   
85.775000   37    -6.921013  0.153846 -6.767167 -6.898067  1.572664e-01   
109.550000  38    -6.945043  0.197564 -6.747480 -6.928809  2.050754e-01   
133.325000  39    -6.972105  0.239679 -6.732425 -6.964140  2.521234e-01   
157.100000  40    -7.004073  0.280161 -6.723912 -7.005712  2.978988e-01   
180.875000  41    -7.041552  0.319618 -6.721934 -7.054020  3.423443e-01   
204.650000  42    -7.085324  0.358319 -6.727005 -7.109760  3.853907e-01   
228.425000  43    -7.136296  0.396141 -6.740155 -7.173764  4.267916e-01   
252.200000  44    -7.195940  0.433132 -6.762809 -7.247175  4.665794e-01   
275.975000  45    -7.266746  0.469455 -6.797291 -7.331708  5.049731e-01   
299.750000  46    -7.352177  0.506072 -6.846106 -7.429054  5.428791e-01   
323.525000  47    -7.450765  0.549087 -6.901678 -7.536596  5.853903e-01   
347.300000  48    -7.555745  0.606231 -6.949514 -7.648361  6.395112e-01   
371.075000  49    -7.662868  0.679134 -6.983734 -7.760653  7.078338e-01   
394.850000  50    -7.773805  0.770544 -7.003262 -7.875038  7.936461e-01   
418.625000  51    -7.889954  0.883829 -7.006124 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')